# Use LangChain with Meta Llama 3 in Azure AI and Azure ML

You can use Meta-Llama-3 models deployed in Azure AI and Azure ML with `langchain` to create more sophisticated intelligent applications. Use `langchain_community` package with the Azure Machine Learning integration.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select Meta-Llama-3 models from Model catalog. This example assumes you are deploying `Meta-Llama-3-70B-Instruct`.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://aka.ms/meta-llama-3-azure-ai-studio-docs) for model deployment and inference.

To complete this tutorial, you will need to:

* Install `langchain` and `langchain_community`:

    ```bash
    pip install langchain langchain_community
    ```

## Example

The following example demonstrate how to create a chain that uses a Meta-Llama-3 chat model deployed in Azure AI and Azure ML. The chain has been configured with a `ConversationBufferMemory`. This example has been adapted from [LangChain official documentation](https://python.langchain.com/docs/modules/memory/adding_memory).

In [33]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    LlamaChatContentFormatter,
)

from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
load_dotenv()

True

Let's create an instance of our `AzureMLChatOnlineEndpoint` model. This class allow us to get access to any model deployed in Azure AI or Azure ML. For completion models use class `langchain_community.llms.azureml_endpoint.AzureMLOnlineEndpoint` with `LlamaContentFormatter` as the `content_formatter`.

In [30]:
chat_model = AzureMLChatOnlineEndpoint(
    endpoint_url="https://mlw-dp100-labs-wwyxg.eastus.inference.ml.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    content_formatter=LlamaChatContentFormatter(),
    max_tokens=600
)

/home/shreyak/programming/azure-ai-learning/venv/lib/python3.10/site-packages/langchain_community/chat_models/azureml_endpoint.py:180: UserWarning: `LlamaChatContentFormatter` will be deprecated in the future. 
                Please use `CustomOpenAIChatContentFormatter` instead.  
            
  warnings.warn(


In [38]:
user_input_prompt = "can machine learning be used to predict when to sell and buy a stock depending on last 18 years data?"

> Tip: You can configure environment variables `AZUREML_ENDPOINT_URL`, `AZUREML_ENDPOINT_API_KEY`, and `AZUREML_ENDPOINT_API_TYPE` instead of passing them as arguments.

In [39]:

response = chat_model.invoke(
    [HumanMessage(content=user_input_prompt)],
    max_tokens=512,
)
print(response.content)

What a great question!

In principle, yes, machine learning can be used to predict when to sell and buy a stock, given a large dataset of historical stock prices and other relevant factors. This is a classic problem in finance, known as Technical Analysis or Trend Analysis.

Here's a high-level overview of how you could approach this:

**Data preprocessing**

1. Collect a large dataset of historical stock prices for the stock you're interested in, covering at least 18 years.
2. Preprocess the data by:
	* Handling missing values or irregularly spaced data.
	* Normalizing the data (e.g., scaling or standardizing) to ensure features have similar magnitudes.
	* Extracting relevant features from the data, such as:
		+ Moving averages (e.g., 50-day, 100-day).
		+ Relative strength index (RSI).
		+ Bollinger Bands.
		+ Volume indicators (e.g., On Balance Volume).
		+ Other technical indicators you think might be relevant.

**Feature engineering**

1. Design a set of features that capture the 

Invoke directly

In the below prompt, we have two input keys: one for the actual input (`human_input`), and another for the input from the `Memory` class (`chat_history`).

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. Be detail and action oriented in your responses and act like an expert in computer science and financial markets"
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

We create the chain as follows:

In [29]:
chat_llm_chain.predict(human_input=user_input_prompt)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human. Be detail and action oriented in your responses and act like an expert in computer science and financial markets
Human: can machine learning be used to predict when to sell and buy a stock depending on last 18 years data?

> Finished chain.


'What a fascinating question!\n\nYes, machine learning can be used to predict when to'

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)
